# Setup Trustee

Since we still are in the user secure environment, start Trustee server using a prepared container image at `quay.io/confidential-devhub/trustee-server` with the right parameters.

## Create kbs-config.toml

Set up basic configs like the http(s) server, which key to trust and so on.

In [ ]:
%%bash

MACHINE_IP_ADDR=<your_trustee_ip_address>

cat > kbs-config.toml <<EOF
[http_server]
sockets = ["127.0.0.1:8080","$MACHINE_IP_ADDR:8080"]
insecure_http = false #change this to true if no https is needed
private_key = "/https_private_key" # if https is not needed, comment this line out
certificate = "/https_certificate" # if https is not needed, comment this line out

[admin]
insecure_api = true
auth_public_key = "/etc/auth-secret/publicKey"

[attestation_token]
insecure_key = true
attestation_token_type = "CoCo"

[attestation_service]
type = "coco_as_builtin"
work_dir = "/opt/confidential-containers/attestation-service"
policy_engine = "opa"

  [attestation_service.attestation_token_broker]
  type = "Ear"
  policy_dir = "/opt/confidential-containers/attestation-service/policies"

  [attestation_service.attestation_token_config]
  duration_min = 5

  [attestation_service.rvps_config]
  type = "BuiltIn"

    [attestation_service.rvps_config.storage]
    type = "LocalJson"
    file_path = "/opt/confidential-containers/rvps/reference-values/reference-values.json"

[[plugins]]
name = "resource"
type = "LocalFs"
dir_path = "/opt/confidential-containers/kbs/repository"

[policy_engine]
policy_path = "/opt/confidential-containers/opa/policy.rego"
EOF

## Generate the authentication key

This key is used for authenticating with the server and installing secrets into it. This means that only clients with the private key counterpart are able to insert secrets into Trustee.

In [ ]:
%%bash

openssl genpkey -algorithm ed25519 auth_private.key
openssl pkey -in auth_private.key -pubout -out auth_public.pub

## Optional configs

For example, the Trustee server defaults to empty reference values `[]`. By creating `reference-values.json` we can provide actual expected measurements. To keep this demo simple, we will not provide them.

## Run the Trustee server

Run the container image `quay.io/confidential-devhub/trustee-server`.

In [ ]:
%%bash

podman run \
  --privileged \
  -v ./kbs-config.toml:/etc/kbs-config/kbs-config.toml \
  -v ./auth_public.pub:/etc/auth-secret/publicKey \
  -v /dev/log:/dev/log \
  -p 8080:8080 \
  --rm \
  quay.io/confidential-devhub/trustee-server:v0.3.0 \
  /usr/local/bin/kbs --config-file /etc/kbs-config/kbs-config.toml

If the https self signed certificate has been added, extend the podman command with `  -v ./https_private_key:/https_private_key
  -v ./https_certificate:/https_certificate`. If the reference values are provided, extend the podman command with `-v ./reference-values.json:/opt/confidential-containers/rvps/reference-values/reference-values.json`.

## Insert the dataset key into Trustee

Insert `key.bin` as secret in Trustee located at path `default/fraud-detection/key.bin`.

In [ ]:
%%bash

sudo podman run --network=host \
  --privileged \
  --device /dev/tpm0 \
  -v ./auth_private.key:/private-key \
  -v ./key.bin:/key.bin \
  -v /dev/log:/dev/log \
  --rm \
  quay.io/confidential-containers/kbs-client:v0.3.0 \
  kbs-client --url http://localhost:8080 config --auth-private-key private-key set-resource --resource-file key.bin --path default/fraud-detection/key.bin

Succesful output will be

```
Set resource success
 resource: <base64 of your key>
 ```